In [ ]:
#https://www.kaggle.com/isaienkov/ingv-volcanic-eruption-prediction-eda-modeling/notebook
#https://www.kaggle.com/jesperdramsch/introduction-to-volcanology-seismograms-and-lgbm/data

In [1]:
#import libraries
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split,KFold
import optuna
from optuna.samplers import TPESampler

In [2]:
os.chdir("../input/predict-volcanic-eruptions-ingv-oe")
print(os.getcwd())

/kaggle/input/predict-volcanic-eruptions-ingv-oe


# Data Overview

In [3]:
train=pd.read_csv('train.csv')
train.head()

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429


In [4]:
train_files=glob.glob("train/*")
print("Number of train files: "+str(len(train_files)))
test_files=glob.glob("test/*")
print("Number of test files: "+str(len(test_files)))

Number of train files: 4431
Number of test files: 4520


# Modeling

In [5]:
def statistics(file_directory):
    df=pd.read_csv(file_directory)
    df=df.fillna(0)
    df=df.agg(['min','std',"kurtosis"])#https://www.kaggle.com/jesperdramsch/introduction-to-volcanology-seismograms-and-lgbm/dataから直感
    df_flat=df.stack()
    df_flat.index=df_flat.index.map('{0[1]}_{0[0]}'.format)
    df_out=df_flat.to_frame().T
    df_out["segment_id"]=int(file_directory.split("/")[1].split(".")[0])
    return df_out

train statsを作成する

In [6]:
train_stats=pd.DataFrame()
for train_file in tqdm(train_files):
    train_stats=pd.concat([train_stats,statistics(train_file)])
    
train_stats.head()

100%|██████████| 4431/4431 [12:02<00:00,  6.13it/s]


,sensor_1_min,sensor_2_min,sensor_3_min,sensor_4_min,sensor_5_min,sensor_6_min,sensor_7_min,sensor_8_min,sensor_9_min,sensor_10_min,...,sensor_2_kurtosis,sensor_3_kurtosis,sensor_4_kurtosis,sensor_5_kurtosis,sensor_6_kurtosis,sensor_7_kurtosis,sensor_8_kurtosis,sensor_9_kurtosis,sensor_10_kurtosis,segment_id
0,-3891.0,-28726.0,0.0,-5282.0,-7223.0,-6189.0,-2455.0,-4704.0,0.0,-10709.0,...,246.222695,0.000000,15.248723,120.483182,-0.114774,0.361385,0.106939,0.000000,11.990491,800654756
0,-2165.0,-3842.0,-2484.0,-2289.0,-1235.0,-2661.0,-5923.0,-3654.0,-3032.0,-4234.0,...,0.948295,0.173209,0.318055,0.785101,0.113250,2.021522,0.901417,0.200506,0.510147,321543978
0,-1242.0,0.0,-1432.0,-2027.0,-1338.0,-6108.0,-3344.0,0.0,-1496.0,-3299.0,...,0.000000,1.560024,1.675906,2.598743,4.266276,1.914384,0.000000,1.297561,1.433292,1417547769
0,-2310.0,-10744.0,-2177.0,-3033.0,-1772.0,-6811.0,-4120.0,-4488.0,-2312.0,-4398.0,...,2.513312,-0.006577,-0.218746,0.940171,0.077095,-0.217828,-0.276764,0.252821,-0.079585,729870090
0,-2717.0,-5526.0,-2822.0,-3154.0,-1458.0,-2280.0,-2151.0,-2298.0,-2154.0,-4081.0,...,9.583151,3.931743,4.876456,3.647493,0.044387,1.969521,0.286740,2.108477,3.026823,830695026


test statsを作成する

In [7]:
test_stats=pd.DataFrame()
for test_file in tqdm(test_files):
    test_stats=pd.concat([test_stats,statistics(test_file)])
    
test_stats.head()

100%|██████████| 4520/4520 [12:01<00:00,  6.27it/s]


,sensor_1_min,sensor_2_min,sensor_3_min,sensor_4_min,sensor_5_min,sensor_6_min,sensor_7_min,sensor_8_min,sensor_9_min,sensor_10_min,...,sensor_2_kurtosis,sensor_3_kurtosis,sensor_4_kurtosis,sensor_5_kurtosis,sensor_6_kurtosis,sensor_7_kurtosis,sensor_8_kurtosis,sensor_9_kurtosis,sensor_10_kurtosis,segment_id
0,-2035.0,0.0,-1627.0,-2429.0,-1534.0,-2439.0,-1991.0,-2281.0,-2373.0,-3401.0,...,0.000000,0.134217,0.217595,1.128608,-0.089367,-0.112290,0.047616,0.599324,0.311241,473253715
0,-2101.0,-32767.0,-1932.0,-3179.0,-3604.0,-4119.0,-2451.0,-4816.0,-2643.0,-4645.0,...,31.380589,1.117916,2.829589,10.087975,1.976502,0.545730,0.674631,1.549404,2.389208,698018079
0,-830.0,0.0,0.0,-877.0,-734.0,-1290.0,-1299.0,-1583.0,-996.0,0.0,...,0.000000,0.000000,0.258753,0.440732,0.459645,0.654342,-0.030561,0.425849,0.000000,1102809614
0,-1078.0,-2396.0,-980.0,-1715.0,-673.0,-14409.0,-2570.0,-3952.0,-1915.0,-2167.0,...,0.938544,0.388946,1.029451,0.606247,12.969725,0.919708,1.255523,0.829786,0.168186,1087681649
0,-2004.0,0.0,-1338.0,-2960.0,-2087.0,-1114.0,-989.0,0.0,-2122.0,-4229.0,...,0.000000,1.630364,3.908271,6.960274,0.030785,0.840082,0.000000,3.120450,4.527737,1939361933


train_statsからtrain_setを作成する

In [8]:
df=train_stats.copy()
df=df.merge(train,on="segment_id")
train_set=df.drop(["segment_id"], axis=1)
train_set.head()

,sensor_1_min,sensor_2_min,sensor_3_min,sensor_4_min,sensor_5_min,sensor_6_min,sensor_7_min,sensor_8_min,sensor_9_min,sensor_10_min,...,sensor_2_kurtosis,sensor_3_kurtosis,sensor_4_kurtosis,sensor_5_kurtosis,sensor_6_kurtosis,sensor_7_kurtosis,sensor_8_kurtosis,sensor_9_kurtosis,sensor_10_kurtosis,time_to_eruption
0,-3891.0,-28726.0,0.0,-5282.0,-7223.0,-6189.0,-2455.0,-4704.0,0.0,-10709.0,...,246.222695,0.000000,15.248723,120.483182,-0.114774,0.361385,0.106939,0.000000,11.990491,16818516
1,-2165.0,-3842.0,-2484.0,-2289.0,-1235.0,-2661.0,-5923.0,-3654.0,-3032.0,-4234.0,...,0.948295,0.173209,0.318055,0.785101,0.113250,2.021522,0.901417,0.200506,0.510147,10340827
2,-1242.0,0.0,-1432.0,-2027.0,-1338.0,-6108.0,-3344.0,0.0,-1496.0,-3299.0,...,0.000000,1.560024,1.675906,2.598743,4.266276,1.914384,0.000000,1.297561,1.433292,40087733
3,-2310.0,-10744.0,-2177.0,-3033.0,-1772.0,-6811.0,-4120.0,-4488.0,-2312.0,-4398.0,...,2.513312,-0.006577,-0.218746,0.940171,0.077095,-0.217828,-0.276764,0.252821,-0.079585,31317486
4,-2717.0,-5526.0,-2822.0,-3154.0,-1458.0,-2280.0,-2151.0,-2298.0,-2154.0,-4081.0,...,9.583151,3.931743,4.876456,3.647493,0.044387,1.969521,0.286740,2.108477,3.026823,5801334


test_statsからtest_setを作成する(サンプル提出時に使う)

In [9]:
df1=test_stats.copy()
test_set=df1.drop(["segment_id"], axis=1)
test_set.head()

,sensor_1_min,sensor_2_min,sensor_3_min,sensor_4_min,sensor_5_min,sensor_6_min,sensor_7_min,sensor_8_min,sensor_9_min,sensor_10_min,...,sensor_1_kurtosis,sensor_2_kurtosis,sensor_3_kurtosis,sensor_4_kurtosis,sensor_5_kurtosis,sensor_6_kurtosis,sensor_7_kurtosis,sensor_8_kurtosis,sensor_9_kurtosis,sensor_10_kurtosis
0,-2035.0,0.0,-1627.0,-2429.0,-1534.0,-2439.0,-1991.0,-2281.0,-2373.0,-3401.0,...,0.170257,0.000000,0.134217,0.217595,1.128608,-0.089367,-0.112290,0.047616,0.599324,0.311241
0,-2101.0,-32767.0,-1932.0,-3179.0,-3604.0,-4119.0,-2451.0,-4816.0,-2643.0,-4645.0,...,1.160786,31.380589,1.117916,2.829589,10.087975,1.976502,0.545730,0.674631,1.549404,2.389208
0,-830.0,0.0,0.0,-877.0,-734.0,-1290.0,-1299.0,-1583.0,-996.0,0.0,...,0.256339,0.000000,0.000000,0.258753,0.440732,0.459645,0.654342,-0.030561,0.425849,0.000000
0,-1078.0,-2396.0,-980.0,-1715.0,-673.0,-14409.0,-2570.0,-3952.0,-1915.0,-2167.0,...,0.188954,0.938544,0.388946,1.029451,0.606247,12.969725,0.919708,1.255523,0.829786,0.168186
0,-2004.0,0.0,-1338.0,-2960.0,-2087.0,-1114.0,-989.0,0.0,-2122.0,-4229.0,...,3.527613,0.000000,1.630364,3.908271,6.960274,0.030785,0.840082,0.000000,3.120450,4.527737


train_setの説明変数(X)と目的変数(y)を分割

In [10]:
X=train_set[train_set.columns.drop("time_to_eruption")]
y=train_set["time_to_eruption"]

↓特徴量の数を減らすことができる。ここではしない。[](http://)

In [ ]:
#For Future selection
#Recurive Feature Elimination #https://qiita.com/rockhopper/items/a68ceb3248f2b3a41c89
selector=RFE(RandomForestRegressor(n_estimators=100,random_state=42),n_features_to_select=28)
selector.fit(X,y)
mask=selector.get_support()#Only saved features will be "True".

print(mask)

selectedX=X.copy()
selectedX=selectedX.loc[:, mask]
selectedX.head()

train/validation/testに分割

In [11]:
#train/validation/test=64%/16%/20%
train_x, test_x, train_y, test_y=train_test_split(X,y,random_state=42, test_size=0.2,shuffle=True)
train_x, val_x, train_y, val_y=train_test_split(train_x,train_y,random_state=42, test_size=0.2,shuffle=True)

In [31]:
#LGBMRegressor + optuna
sampler=TPESampler(seed=42)

def create_model_lgbm(trial):
    num_leaves=trial.suggest_int("num_leaves",2,31)
    n_estimators=trial.suggest_int("n_estimators",50,300)
    max_depth=trial.suggest_int("max_depth",3,8)
    min_child_samples=trial.suggest_int("min_child_samples",100,1200)
    learning_rate=trial.suggest_uniform("learning_rate",0.0001,0.99)
    min_data_in_leaf=trial.suggest_int('min_data_in_leaf',5,90)
    feature_fraction=trial.suggest_uniform('feature_fraction',0.0001,1.0)
    
    model=lgbm.LGBMRegressor(
    num_leaves=num_leaves,
    n_estimators=n_estimators,
    max_depth=max_depth,
    min_child_samples=min_child_samples,
    min_data_in_leaf=min_data_in_leaf,
    learning_rate=learning_rate,
    feature_fraction=feature_fraction,
    random_state=42)
    
    return model


#目的関数
def objective_lgbm(trial):
    model=create_model_lgbm(trial)
    model.fit(train_x,train_y)
    preds=model.predict(val_x)
    score=mean_absolute_error(val_y,preds)
    return score


study=optuna.create_study(direction="minimize", sampler=sampler)
optuna.logging.disable_default_handler()#don't show log
study.optimize(objective_lgbm, n_trials=100)


#最適解
print(study.best_params)
print(study.best_value)
#print(study.best_trial)

{'num_leaves': 29, 'n_estimators': 195, 'max_depth': 8, 'min_child_samples': 1055, 'learning_rate': 0.08666416859467335, 'min_data_in_leaf': 9, 'feature_fraction': 0.36363854117398253}
5146375.397861516


testで確かめてみよう

In [32]:
#check
params_lgbm= study.best_params

model=lgbm.LGBMRegressor(**params_lgbm)
model.fit(train_x, train_y)
preds=model.predict(test_x)
print('test LGB model mean_squared_error: ', mean_absolute_error(test_y, preds))#over-fitting?

test LGB model mean_squared_error:  4975061.278831848


↑ スコアはまあまあよさそうな感じだけど、過学習気味...

In [ ]:
#サンプル数少ない時は、K分割交差しよう。
n_fold = 7
folds=KFold(n_splits=n_fold, shuffle=True,random_state=42)

params_lgbm=study.best_params

for n_fold, (train_id,val_id) in enumerate(folds.split(selectedX)):
    train_x,train_y= selectedX.iloc[train_id], y.iloc[train_id]
    val_x,val_y= selectedX.iloc[val_id],y.iloc[val_id]
    
    model=lgbm.LGBMRegressor(**params_lgbm)
    model.fit(train_x, train_y)
    preds=model.predict(val_x)
    print('Optimized LGB model mean_squared_error: ', mean_absolute_error(val_y, preds))#over-fitting?
    #https://www.kaggle.com/jesperdramsch/introduction-to-volcanology-seismograms-and-lgbm/data

In [33]:
#XGBRegressor+optuna
import xgboost as xgb

#LGBMRegressor + optuna
sampler=TPESampler(seed=42)

def create_model_xgb(trial):
    n_estimators=trial.suggest_int("n_estimators",50,300)
    max_depth=trial.suggest_int("max_depth",3,15)
    learning_rate=trial.suggest_uniform("learning_rate",0.0001,0.99)
    gamma=trial.suggest_uniform('min_data_in_leaf',0,1)
    model=xgb.XGBRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    learning_rate=learning_rate,
    gamma=gamma,
    random_state=42)
    
    return model


#目的関数
def objective_xgb(trial):
    model1=create_model_xgb(trial)
    model1.fit(train_x,train_y)
    preds=model1.predict(val_x)
    score=mean_absolute_error(val_y,preds)
    return score


study1=optuna.create_study(direction="minimize", sampler=sampler)
optuna.logging.disable_default_handler()#don't show log
study1.optimize(objective_xgb, n_trials=100)

#最適解
print(study1.best_params)
print(study1.best_value)
print(study1.best_trial)

{'n_estimators': 227, 'max_depth': 10, 'learning_rate': 0.08524821682956002, 'min_data_in_leaf': 0.9938134807456179}
4717754.635534203
FrozenTrial(number=86, value=4717754.635534203, datetime_start=datetime.datetime(2020, 12, 31, 7, 39, 0, 250520), datetime_complete=datetime.datetime(2020, 12, 31, 7, 39, 2, 763143), params={'n_estimators': 227, 'max_depth': 10, 'learning_rate': 0.08524821682956002, 'min_data_in_leaf': 0.9938134807456179}, distributions={'n_estimators': IntUniformDistribution(high=300, low=50, step=1), 'max_depth': IntUniformDistribution(high=15, low=3, step=1), 'learning_rate': UniformDistribution(high=0.99, low=0.0001), 'min_data_in_leaf': UniformDistribution(high=1, low=0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=86, state=TrialState.COMPLETE)


testで確かめてみよう

In [34]:
#check
params_xgb=study1.best_params
params_xgb.pop("min_data_in_leaf")
params_xgb

model1=xgb.XGBRegressor(**params_xgb)
model1.fit(train_x, train_y)
preds1=model1.predict(test_x)
print('test LGB model mean_squared_error: ', mean_absolute_error(test_y, preds1))#over-fitting?

test LGB model mean_squared_error:  4710809.221850338


↑いい感じでは？

In [ ]:
##サンプル数少ない時は、K分割交差しよう。
n_fold = 7
folds=KFold(n_splits=n_fold, shuffle=True,random_state=42)


for n_fold, (train_id,val_id) in enumerate(folds.split(selectedX)):
    train_x,train_y= selectedX.iloc[train_id], y.iloc[train_id]
    val_x,val_y= selectedX.iloc[val_id],y.iloc[val_id]
    
    model1=xgb.XGBRegressor(**params_xgb)
    model1.fit(train_x, train_y)
    preds=model.predict(val_x)
    print('Optimized LGB model mean_squared_error: ', mean_absolute_error(val_y, preds))#over-fitting?
    #https://www.kaggle.com/jesperdramsch/introduction-to-volcanology-seismograms-and-lgbm/data

LGBM or XGB でまよったので、いっそ合体してみては...?

In [35]:
preds=model.predict(test_x) #LGBM
preds1=model1.predict(test_x) #XGB

preds2=preds*0.5+preds1*0.5
print('test LGB model mean_squared_error: ', mean_absolute_error(test_y, preds2))#over-fitting?

test LGB model mean_squared_error:  4706492.321967858


↓ニューラルネットもやってみたけど、全然だめだった...

In [ ]:
#newral network regressor
import tensorflow as tf

def nn_model():
    model=tf.keras.Sequential([
    tf.keras.layers.Input((28,)),
    tf.keras.layers.Dense(1000,activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation="relu")
    ])
    
    model.compile(loss="mae",
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))
    
    return model

In [ ]:
n_fold = 3
folds=KFold(n_splits=n_fold, shuffle=True,random_state=42)


for n_fold, (train_id,val_id) in enumerate(folds.split(selectedX)):
    train_x,train_y= selectedX.iloc[train_id], y.iloc[train_id]
    val_x,val_y= selectedX.iloc[val_id],y.iloc[val_id]
    
    model=nn_model()
    
    history=model.fit(train_x, train_y,validation_split=0.1,verbose=0,epochs=1000,batch_size=128)    
    preds=model.predict(val_x)
    
    print('Optimized NN model mean_squared_error: ', mean_absolute_error(val_y, preds))

# Sumbission

In [36]:
test_set.head()

,sensor_1_min,sensor_2_min,sensor_3_min,sensor_4_min,sensor_5_min,sensor_6_min,sensor_7_min,sensor_8_min,sensor_9_min,sensor_10_min,...,sensor_1_kurtosis,sensor_2_kurtosis,sensor_3_kurtosis,sensor_4_kurtosis,sensor_5_kurtosis,sensor_6_kurtosis,sensor_7_kurtosis,sensor_8_kurtosis,sensor_9_kurtosis,sensor_10_kurtosis
0,-2035.0,0.0,-1627.0,-2429.0,-1534.0,-2439.0,-1991.0,-2281.0,-2373.0,-3401.0,...,0.170257,0.000000,0.134217,0.217595,1.128608,-0.089367,-0.112290,0.047616,0.599324,0.311241
0,-2101.0,-32767.0,-1932.0,-3179.0,-3604.0,-4119.0,-2451.0,-4816.0,-2643.0,-4645.0,...,1.160786,31.380589,1.117916,2.829589,10.087975,1.976502,0.545730,0.674631,1.549404,2.389208
0,-830.0,0.0,0.0,-877.0,-734.0,-1290.0,-1299.0,-1583.0,-996.0,0.0,...,0.256339,0.000000,0.000000,0.258753,0.440732,0.459645,0.654342,-0.030561,0.425849,0.000000
0,-1078.0,-2396.0,-980.0,-1715.0,-673.0,-14409.0,-2570.0,-3952.0,-1915.0,-2167.0,...,0.188954,0.938544,0.388946,1.029451,0.606247,12.969725,0.919708,1.255523,0.829786,0.168186
0,-2004.0,0.0,-1338.0,-2960.0,-2087.0,-1114.0,-989.0,0.0,-2122.0,-4229.0,...,3.527613,0.000000,1.630364,3.908271,6.960274,0.030785,0.840082,0.000000,3.120450,4.527737


In [46]:
#まずは、test_setを学習済みXGBRegressorへ

preds=model.predict(test_set) #LGBM
preds1=model1.predict(test_set) #XGB

preds2=preds*0.5+preds1*0.5

In [47]:
test_stats['time_to_eruption']=preds2

In [48]:
sample_submission=pd.read_csv("sample_submission.csv")

In [49]:
sample_submission=pd.merge(sample_submission,test_stats[["segment_id","time_to_eruption"]], on="segment_id" )

In [50]:
sample_submission.head()

,segment_id,time_to_eruption_x,time_to_eruption_y
0,1000213997,0,2.188855e+07
1,100023368,0,3.620270e+07
2,1000488999,0,2.837889e+07
3,1001028887,0,2.392353e+07
4,1001857862,0,2.237853e+07


In [51]:
sample_submission=sample_submission.drop(['time_to_eruption_x'],axis=1)
sample_submission.columns=['segment_id',"time_to_eruption"]
sample_submission

,segment_id,time_to_eruption
0,1000213997,2.188855e+07
1,100023368,3.620270e+07
2,1000488999,2.837889e+07
3,1001028887,2.392353e+07
4,1001857862,2.237853e+07
...,...,...
4515,996704281,2.210747e+07
4516,997630809,2.054629e+07
4517,998072137,2.186448e+07
4518,998136924,3.074510e+07


In [45]:
sample_submission.to_csv("/kaggle/working/submission5.csv", index=False)